# Burn Scar Detection From NOAA-20 VIIRS

## Define Functions to Read in VIIRS ref and geo files 

In [1]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import os

def get_VJ103_geo(geo_file):
    '''
    input: VIIRS VJ103 .nc file
    return: lat, lon, SZA, VZA, SAA, VAA
    '''
    from netCDF4 import Dataset
    
    with Dataset(geo_file, 'r') as nc_geo_file_obj:
        geolocation_ncObj = nc_geo_file_obj['geolocation_data']
        lat, lon = geolocation_ncObj['latitude'][:], geolocation_ncObj['longitude'][:]        
        SZA = geolocation_ncObj['solar_zenith'][:]
        VZA = geolocation_ncObj['sensor_zenith'][:]
        SAA = geolocation_ncObj['solar_azimuth'][:]
        VAA = geolocation_ncObj['sensor_azimuth'][:]
    
    return lat, lon, SZA, VZA, SAA, VAA

def get_VJ102_ref(ref_file):
    '''
    input: VIIRS VJ103 .nc file
    return: lat, lon, SZA, VZA, SAA, VAA
    '''
    from netCDF4 import Dataset
    
    with Dataset(ref_file, 'r') as nc_ref_file_obj:
        observation_data_ncObj = nc_ref_file_obj['observation_data']
        
        M_bands                    = np.zeros((3248,3200,16))
        M_band_solar_irradiances = np.ones(16)
#         M_band_quality_flags  = np.zeros((3232,3200,16))
        for i in range(16):
            i_ = i+1
#             M_band_quality_flags[:,:,i] = observation_data_ncObj['M{:02d}_quality_flags'.format(i_)][:]
            M_bands_temp = observation_data_ncObj['M{:02d}'.format(i_)][:]
            M_band_shape = np.shape(M_bands_temp)
            M_bands[:M_band_shape[0],:,i] = M_bands_temp
            M_bands[M_band_shape[0]:,:,i] = np.nan
#             try:
#                 M_bands[:,:,i]     = M_bands_temp
#             except:
#                 shape
#                 M_bands[:3216,:,i] = M_bands_temp
#                 M_bands[3216:,:,i] = np.nan
                
#                 (3248,3200) into shape (3232,3200)
            try:
                M_bands_rad_scale_factor      = observation_data_ncObj['M{:02d}'.format(i_)].radiance_scale_factor
                M_bands_ref_scale_factor      = observation_data_ncObj['M{:02d}'.format(i_)].scale_factor
                M_band_solar_irradiances[i]   = M_bands_rad_scale_factor/M_bands_ref_scale_factor
#                 print(M_band_solar_irradiances[i])
                M_bands[M_bands >=65532 ]     = np.nan
#                 M_bands[:,:,i]                *= M_bands_ref_scale_factor
                
            except:
                M_bands[M_bands >=65532 ] = np.nan
            
        return M_bands, M_band_solar_irradiances
    
    
    
# VIIRS JPSS-1 naming convention
# 'VJ102MOD.A2021025.1742.021.2021072143738.nc'
# 'VJ103MOD.A2021025.1742.021.2021072125458.nc'
# <product short name>.<aquisition date YYYYDDD>.<UTC HHMM>.<collection version>.<production date YYYYDDDHHMMSS>.nc

# VJ102 => L1B calibrated radiances, 16 M bands, quality flag, 6 minute granules, 3232x3200 pixels
# VJ103 => L1B terrain-corrected geolocation, lat/lon, SZA/VZA/VAA/SAA, land water mask, 
#          quality flag, 



ref_filepath_home = "R:/VIIRS_data/VJ102_ref/"
geo_filepath_home = "R:/VIIRS_data/VJ103_geolocation/"

ref_filepaths = np.array([ref_filepath_home + x for x in os.listdir(ref_filepath_home)])
geo_filepaths = np.array([geo_filepath_home + x for x in os.listdir(geo_filepath_home)])

# sort files by time stamp; after sort check time stamps match up between files
# this will will allow looping between two file lists to access coincident geo and ref
# data wihtout the overhead of check the timestamp match
time_stamps_sorted_idx = np.argsort([x[-33:-21] for x in ref_filepaths])
time_stamps = np.sort([x[-33:-21] for x in ref_filepaths])

ref_filepaths = ref_filepaths[time_stamps_sorted_idx]
geo_filepaths = geo_filepaths[time_stamps_sorted_idx]
print(ref_filepaths[0][-33:-21])
# print([x[-33:-21] for x in ref_filepaths] == [x[-33:-21] for x in geo_filepaths])

2021025.1742


In [2]:
from netCDF4 import Dataset
# geo_file = geo_filepaths[2]
# with Dataset(geo_file, 'r') as nc_geo_file_obj:
#     geolocation_ncObj = nc_geo_file_obj['geolocation_data']
# #     print(nc_geo_file_obj['navigation_data'])
#     #201 scan lines, each scan line is 16 pixel => 3216 pixels across all scan line
#     earth_sun_distance_per_scanline = nc_geo_file_obj['navigation_data']['earth_sun_distance'][:]

test_ref="R:/VIIRS_data/test_case_burn_scar/VJ102MOD.A2021287.2054.021.2021288023526.nc"
test_geo="R:/VIIRS_data/test_case_burn_scar/VJ103MOD.A2021287.2054.021.2021288021701.nc"

ref_file = "R:/VIIRS_data/test_case_burn_scar/VJ102MOD.A2021289.2018.021.2021290024014.nc"#ref_filepaths[2]
print(ref_file)
with Dataset(ref_file, 'r') as nc_ref_file_obj:
    print(nc_ref_file_obj)
    ref_ncObj = nc_ref_file_obj['observation_data']
    print(ref_ncObj['M01'])

R:/VIIRS_data/test_case_burn_scar/VJ102MOD.A2021289.2018.021.2021290024014.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    date_created: 2021-10-16T22:40:48Z
    ProductionTime: 2021-10-16T22:40:48Z
    title: VIIRS M-band Reflected Solar Band and Thermal Emissive Band Data
    ShortName: VJ102MOD
    LongName: VIIRS/JPSS1 Moderate Resolution 6-Min L1B Swath 750m
    instrument: VIIRS
    processing_version: v3.0.0
    Conventions: CF-1.6
    institution: NASA Goddard Space Flight Center, VIIRS L1 Processing Group
    license: http://science.nasa.gov/earth-science/earth-science-data/data-information-policy/
    naming_authority: gov.nasa.gsfc.VIIRSland
    keywords_vocabulary: NASA Global Change Master Directory (GCMD) Science Keywords
    standard_name_vocabulary: 
    creator_name: VIIRS L1 Processing Group
    creator_email: modis-ops@lists.nasa.gov
    creator_url: https://ladsweb.modaps.eosdis.nasa.gov
    project: VIIRS L1 Project
    

<ipython-input-2-dc4a2b6097cf>:15: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(nc_ref_file_obj)
<ipython-input-2-dc4a2b6097cf>:17: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(ref_ncObj['M01'])


## Get BRF and Geolocation for Burn Scar Analysis

In [28]:
test_ref="R:/VIIRS_data/test_case_burn_scar/VJ102MOD.A2021287.2054.021.2021288023526.nc"
test_geo="R:/VIIRS_data/test_case_burn_scar/VJ103MOD.A2021287.2054.021.2021288021701.nc"

In [32]:
file_num = -1
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# lat, lon, SZA, VZA, SAA, VAA = get_VJ103_geo(test_geo)#geo_filepaths[file_num])
# test_geo_colorado = "R:/VNP03MOD.A2020290.2012.002.2021126032457.nc"
# test_geo_newmexico = ''
lat, lon, SZA, VZA, SAA, VAA = get_VJ103_geo(test_geo)#geo_filepaths[file_num])

lat, lon  = flip_arr(lat), flip_arr(lon)
# print(geo_filepaths[file_num])
time_stamp_current = test_geo[-33:-21] #geo_filepaths[file_num][-33:-21]

In [31]:
plt.imshow(lat)
plt.show()

In [5]:
# M_bands, M_band_solar_irradiances = get_VJ102_ref(test_ref)#ref_filepaths[file_num])
test_ref_colorado = "R:/VJ102MOD.A2020290.1918.002.2020291003928.nc"
M_bands, M_band_solar_irradiances = get_VJ102_ref(test_ref)#ref_filepaths[file_num])

## BRF RGB

In [6]:
# M_bands, M_band_solar_irradiances = get_VJ102_ref(test_ref)#ref_filepaths[file_num])
# test_ref_colorado = "R:/VNP02MOD.A2020290.2012.002.2021159101437.nc"
# M_bands, M_band_solar_irradiances = get_VJ102_ref(test_ref_colorado)#ref_filepaths[file_num])

%matplotlib qt
# %matplotlib inline
plt.figure(figsize=(15,15))
cosSZA = np.cos(np.deg2rad(SZA))
print(M_bands[:5,:5,5])
# M_bands[M_bands<0]==0
M_bands_norm = np.copy(M_bands)
print(np.nanmean(M_bands_norm))
for i in range(16):
    try:
        M_bands_norm[:,:,i] /=  cosSZA
    except:
        M_bands_norm[:3216,:,i] /=  cosSZA[:3216,:]
#     M_bands_norm[:,:,i] /=np.max(M_bands_norm[:,:,i])
    
rgb= np.dstack((M_bands_norm[:,:,4], M_bands_norm[:,:,3], M_bands_norm[:,:,2]))
rgb=np.flip(rgb, axis=0)
rgb=2.5*np.flip(rgb, axis=1)
plt.imshow(rgb)#, cmap='binary')
# plt.imshow(np.flip(lat, axis=0), cmap='jet')
# plt.colorbar()
plt.show()

[[       nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan]
 [0.11053443 0.11007462 0.10985471 0.10973476 0.10979474]
 [0.11049444 0.10983472 0.11017457 0.11047445 0.10991468]
 [0.1089151  0.10919498 0.10791551 0.10885512 0.10837532]]
1.5913811689609716


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


## Define Functions for NBR and BS RGB

In [7]:

# Grab VIIRS bands (0.86 microns I2 or 0.86 M7) and 2.25 M11
# we can calculate (R225-R86)/(R86+R225)
# then make a threshold cut off for the burn scar area

def normalized_burn_ratio(R_M7, R_M11):
    
    return (R_M11-R_M7)/(R_M11+R_M7)

# burn scar RGB composite <2.1, 0.86, 0.64> = <M11, I2, I1>
# can also do a threshold for burn scars

def burn_scar_RGB_composite(R_M11, R_M7, R_M5):
    
    return np.dstack((R_M11, R_M7, R_M5))

## Normalized Burn Ratio (NBR)

In [8]:
# %matplotlib inline
# %matplotlib qt

from datetime import datetime

R_M7, R_M11 = M_bands_norm[:,:,6], M_bands_norm[:,:,10]
NBR = normalized_burn_ratio(R_M7, R_M11)
plt.style.use('dark_background')
def flip_arr(arr):
    arr=np.flip(arr, axis=0)
    arr=np.flip(arr, axis=1)
    return arr

NBR = flip_arr(NBR)
# lat = flip_arr(lat)
# lon = flip_arr(lon)
    
max_nbr, min_nbr = np.nanmax(NBR), np.nanmin(NBR)
# NBR[np.isnan(NBR)]=-999

#plot
font_size=20
plt.rcParams['font.size'] = font_size


fig, ax = plt.subplots(figsize=(10, 5))

# import cartopy.crs as ccrs
# crs = ccrs.PlateCarree()
# ax = plt.axes(projection=crs)
# ax.coastlines(resolution='10m', color='white',alpha=0.5)

vmin, vmax = -0.9, 0.5
cmap='gist_ncar'#'RdYlGn_r'
# print(np.shape(lat), np.shape(lon), np.shape(NBR))
if np.shape(lat)[0] < np.shape(NBR)[0]:
    NBR=NBR[:np.shape(lat)[0],:]
# NBR = flip_arr(NBR)
# im = ax.pcolormesh(lon,lat, NBR, cmap=cmap, vmin=vmin, vmax=vmax)
im = ax.imshow(NBR, cmap=cmap, vmin=vmin, vmax=vmax)
im.cmap.set_under('white')

# ax.coastlines(resolution='10m', color='black',alpha=0.5)

year = time_stamp_current[:4]
DOY  = '{}'.format(time_stamp_current[4:7])
UTC_hr  = time_stamp_current[8:][:2]
UTC_min = time_stamp_current[8:][2:]
date = datetime.strptime(year + "-" + DOY, "%Y-%j").strftime("%m-%d-%Y")
title = 'Normalized Burn Ratio (NBR)\n VIIRS NOAA-20 {} {}:{} UTC\n(2.25-0.86)/(2.25+0.86) > 0\nmin = {:02.2f}, max = {:02.2f}'.format(date, UTC_hr, UTC_min, min_nbr, max_nbr)
ax.set_title(title)

# cbar = fig.colorbar(im, fraction=0.046, pad=0.04, ticks = np.arange(vmin, vmax+0.1, 0.1))
# cbar.ax.tick_params(labelsize=font_size)

# cb_ax = fig.add_axes([0.93, 0.1, 0.02, 0.8])
cb_ax = fig.add_axes([0.1, 0.05, 0.35, 0.032]) #[left, bottom, width, height]
cbar = fig.colorbar(im, cax=cb_ax, ticks = np.arange(vmin, vmax+0.1, 0.1),orientation="horizontal")

# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#                   linewidth=1, color='gray', alpha=0.5, linestyle=':')

# plt.savefig('./NBR_'+date+'.PDF', format='pdf', dpi=200)
plt.tight_layout()
plt.show()


<ipython-input-8-4f406887c162>:64: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## Burn Scar RGB Composite

In [9]:
# %matplotlib inline
# %matplotlib qt
R_M11, R_M7, R_M5 = M_bands_norm[:,:,10], M_bands_norm[:,:,6], M_bands_norm[:,:,4]
burn_scar_RGB = burn_scar_RGB_composite(R_M11, R_M7, R_M5)
burn_scar_RGB=np.flip(burn_scar_RGB, axis=0)
burn_scar_RGB=np.flip(burn_scar_RGB, axis=1)
plt.figure(figsize=(15,15))
r1,r2, c1,c2 = 240,746, 1235,1836
plt.imshow(2.5*burn_scar_RGB[r1:r2,c1:c2])
plt.tight_layout()
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


## Calc BS RGB + NDVI + NBR + BS Mask

In [10]:
import scipy.misc
from scipy import ndimage
r11, r7 = np.copy(R_M11), np.copy(R_M7)
#NDVI
R_M7, R_M5 = M_bands_norm[:,:,6], M_bands_norm[:,:,4]
NDVI = (R_M7-R_M5)/(R_M7+R_M5)
NDVI=np.flip(NDVI, axis=0)
NDVI=np.flip(NDVI, axis=1)

#NBR
NBR_blurred = ndimage.gaussian_filter(np.copy(NBR), sigma=1)
# NBR_masked = np.copy(NBR)
# NBR_masked[idx_veggie] = np.nan

#burn_scar_RGB_mask
burn_scar_RGB_mask = np.copy(burn_scar_RGB)
# burn_scar_RGB_mask = ndimage.gaussian_filter(burn_scar_RGB_mask, sigma=1)


#construct thresholds for each channel of RGB
thresh_upper_NIR = 0.0758
thresh_upper_idx_NIR = np.where(burn_scar_RGB[:,:,0]>thresh_upper_NIR)   
thresh_lower_NIR = 0.0225
thresh_lower_idx_NIR = np.where(burn_scar_RGB[:,:,0]<thresh_lower_NIR)

thresh_upper_veggie = 0.1346
thresh_upper_idx_veggie = np.where(burn_scar_RGB[:,:,1]>thresh_upper_veggie)   
thresh_lower_veggie = 0.0281
thresh_lower_idx_veggie = np.where(burn_scar_RGB[:,:,1]<thresh_lower_veggie)  

thresh_upper_vis = 0.0328
thresh_upper_idx_vis = np.where(burn_scar_RGB[:,:,2]>thresh_upper_vis)   
thresh_lower_vis = 0.0188
thresh_lower_idx_vis = np.where(burn_scar_RGB[:,:,2]<thresh_lower_vis)

thresh_sand = 0.25
idx_sand = np.where(burn_scar_RGB[:,:,2]>thresh_sand)  
    
for i in range(3):
    
#     burn_scar_RGB_mask[:,:,i][thresh_upper_idx_NIR]    = 0
    burn_scar_RGB_mask[:,:,i][thresh_upper_idx_veggie] = 0
#     burn_scar_RGB_mask[:,:,i][thresh_upper_idx_vis]    = 0
    
#     burn_scar_RGB_mask[:,:,i][thresh_lower_idx_NIR]    = 0
    burn_scar_RGB_mask[:,:,i][thresh_lower_idx_veggie] = 0
#     burn_scar_RGB_mask[:,:,i][thresh_lower_idx_vis]    = 0
    
#     burn_scar_RGB_mask[:,:,i][NDVI>0.32]=0
#     burn_scar_RGB_mask[:,:,i][NBR<-0.25]=0
    
#collopase RGB into one channel where non-zero positve values are valid
valid_idx_burn_scar_RGB_mask = np.where(burn_scar_RGB_mask !=0)
# burn_scar_RGB_mask_collapsed = np.sum(burn_scar_RGB_mask, axis=2) 
# burn_scar_RGB_mask_collapsed[burn_scar_RGB_mask_collapsed==0] = np.nan
burn_scar_RGB_mask_collapsed = burn_scar_RGB_mask[:,:,0]

burn_scar_RGB_mask_collapsed = ndimage.gaussian_filter(burn_scar_RGB_mask_collapsed, sigma=3)
burn_scar_RGB_mask_collapsed[burn_scar_RGB_mask_collapsed<0.01]=np.nan


NBR_composite = (burn_scar_RGB[:,:,0] - burn_scar_RGB[:,:,1])/(burn_scar_RGB[:,:,0] + burn_scar_RGB[:,:,1])
NBR_composite[np.isnan(burn_scar_RGB_mask_collapsed)] = np.nan
print(np.shape(burn_scar_RGB_mask_collapsed))
print(np.shape(burn_scar_RGB))
# burn_scar_RGB_mask = ndimage.gaussian_filter(burn_scar_RGB_mask, sigma=3)
# for i in range(3):
#     #normalize
#     burn_scar_RGB_mask[:,:,i]/=np.nanmax(burn_scar_RGB_mask[:,:,i])
# burn_scar_RGB_mask=burn_scar_RGB_mask[:,:,0]
# burn_scar_RGB_mask[burn_scar_RGB_mask<0.01]=np.nan
# burn_scar_RGB_mask=5*burn_scar_RGB_mask
# burn_scar_RGB_mask[burn_scar_RGB_mask<0.098]=0
# burn_scar_RGB_mask[burn_scar_RGB_mask>0.002] = 1 


(3248, 3200)
(3248, 3200, 3)


## Save RGB, Mask, NBR, NDVI to geotiff for Real Earth case

In [11]:
def write2tif(data, file_path):
    '''
    give data and path name and save to tif file using rasterio
    '''
    import numpy as np
    import rasterio
    
    num_dims = data.ndim
    if num_dims == 3:
        height, width, count = np.shape(data)
        data = np.moveaxis(data, 2,0)
    elif num_dims ==2:
        height, width = np.shape(data)
        #first dim must be count, so reshape
        data = np.reshape(data, (1, height, width))
        count = 1
    else:
        return "invalid dim number, must be (ny, nx, bands) or (ny, nx)"
    
#     dtype = np.dtype(data)
#     print(dtype)
    print(height, width, count)
    
    with rasterio.open('{}.tif'.format(file_path), 'w',\
                   count=count, width=width, height=height,\
                   dtype=data.dtype, driver="GTiff") as src:
        
        src.write(data)    



In [12]:
write2tif(burn_scar_RGB, '../burn_scar_RGB_10_16_2020_colorado_fires')
write2tif(burn_scar_RGB_mask_collapsed, '../burn_scar_mask_10_16_2020_colorado_fires')
write2tif(NBR, '../NBR_10_16_2020_colorado_fires')
write2tif(NDVI, '../NDVI_10_16_2020_colorado_fires')

3248 3200 3


R:\anaconda3\envs\pytaf_test\lib\site-packages\rasterio\__init__.py:238: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = writer(path, mode, driver=driver,


3248 3200 1
3232 3200 1
3248 3200 1


In [13]:
with rasterio.open('../burn_scar_RGB_10_16_2020_colorado_fires.tif', 'r') as src:
#         print(src.crs, src.width)
        im = src.read()
#         im = np.reshape(im, (src.height, src.width))
#         print(im[im<33000])
#         print(src.info)
        im = plt.imshow(1.5*np.moveaxis(im, 0,2))
        plt.show()

NameError: name 'rasterio' is not defined

In [ ]:
#open viirs tif file
%matplotlib qt
filename_viirs_tif_vj102 = r'R:\VJ102MOD.A2020248.1906.002.2020249003749.sg_000501739993-M01.tif'
import os
home = r'R:'
vj102_files = [home+x for x in os.listdir(home) if x[:5]=='VJ102'and x[-3:]=='tif']
# print(vj102_files)
# f=plt.figure(figsize=(13,4))
for tif in vj102_files[:18]:
    with rasterio.open(tif, 'r') as src:
#         print(src.crs, src.width)
        im = src.read()
        im = np.reshape(im, (src.height, src.width))
        print(im[im<33000])
        print(src.info)
#         im = plt.imshow(im, cmap='binary_r')
#         plt.show()
    
#         plt.savefig(r'R:\viirs_tif_images_bank\test{}.png'.format(tif[3:-4]))
        break


## Plot 4-Panel BS RGB + NDVI + NBR + BS Mask

In [ ]:
#plot
# %matplotlib qt
font_size=18
plt.rcParams['font.size'] = font_size
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(25,13), sharex=True, sharey=True)
#sub select region to analyze
# (3216, 3200)
r1,r2, c1,c2 = 0,3216, 0,3200
#colorado SAR case bounds
# r1,r2, c1,c2 = 1575,2183, 1483,2108
# #california dixie fire bounds
r1,r2, c1,c2 = 240,746, 1235,1836
# r1,r2, c1,c2 = 378,563, 1527,1710

cmap='gist_ncar'

ax[1,0].set_title('Day Land Cloud Fire RGB')
im = ax[1,0].imshow(2.5*burn_scar_RGB[r1:r2,c1:c2], cmap=cmap, vmin=vmin, vmax=vmax)

ax[1,1].set_title('Burn Scar Mask')
im1 = ax[1,1].imshow(NBR_composite[r1:r2,c1:c2], cmap=cmap)

# ax[0,1].set_title('NDVI')
# im2 = ax[0,1].imshow(NDVI[r1:r2,c1:c2], cmap=cmap)

# ax[0,0].set_title('NBR')
# im3 = ax[0,0].imshow(NBR[r1:r2,c1:c2], cmap=cmap)

cmap__ = 'gist_stern'
cmap_ = 'jet'
veggie_brf = np.copy(burn_scar_RGB[:,:,1])
ax[0,1].set_title('0.86 micron BRF')
im2 = ax[0,1].imshow(veggie_brf[r1:r2,c1:c2], cmap=cmap_, vmax=0.5)

swir_burn_brf = np.copy(burn_scar_RGB[:,:,0])
# ax[0,0].set_title('2.25 micron BRF')
# im3 = ax[0,0].imshow(swir_burn_brf[r1:r2,c1:c2], cmap=cmap__, vmax=0.8)
ax[0,0].set_title('Normalized Burn Ratio (NBR)')
im3 = ax[0,0].imshow(NBR[r1:r2,c1:c2], cmap=cmap)

print(np.nanmax(veggie_brf[r1:r2,c1:c2]))
print(np.nanmax(swir_burn_brf[r1:r2,c1:c2]))

for a in ax.flat:
    a.set_xticks([])
    a.set_yticks([])

plt.subplots_adjust(left=0.265, bottom=0.012, right=0.770, top=0.908, wspace=0.045, hspace=0)
plt.show()



# im1.cmap.set_under('white')

# ax.set_title(title)

# cbar = fig.colorbar(im, fraction=0.046, pad=0.04, ticks = np.arange(vmin, vmax+0.1, 0.1))
# cbar.ax.tick_params(labelsize=font_size)

### Save Into File for RealEarth Sample

In [76]:
from netCDF4 import Dataset

lat, lon, SZA, VZA, SAA, VAA = get_VJ103_geo(test_geo)#geo_filepaths[file_num])
lat, lon  = flip_arr(lat), flip_arr(lon)
lat_copy = np.copy(lat)
lon_copy = np.copy(lon)

with Dataset('./real_earth_burn_scar_mask_sample_JVB_V1.nc',mode='w') as ncfile:
    
    ncfile.Conventions = 'CF-1.6'
    
    num_rows = r2-r1
    num_cols = c2-c1
    
    lat_dim  = ncfile.createDimension('latitude', num_rows) # latitude axis
    lon_dim  = ncfile.createDimension('longitude', num_cols) # longitude axis
    time_dim = ncfile.createDimension('time', None)    # longitude axis
    
    # Define two variables with the same names as dimensions,
    # a conventional way to define "coordinate variables".
    lat = ncfile.createVariable('lat', np.float32, ('latitude','longitude'), fill_value=-999)
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lat.standard_name = 'latitude'
    lat[:,:] = lat_copy[r1:r2,c1:c2] 
    
    lon = ncfile.createVariable('lon', np.float32, ('latitude','longitude'), fill_value=-999)
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    lon.standard_name = 'longitude'
    lon[:,:] = lon_copy[r1:r2,c1:c2] 
    
    time = ncfile.createVariable('time', np.int8, ('time'))
    time.units = 'hours since 2021-10-14 20:54:00'
    time.long_name = 'time'
    time.calendar = 'none'
    
    # Define a variable to hold the data
    burnscar_composite_with_timeDim = np.empty((1, num_rows, num_cols))
    burnscar_composite_with_timeDim[0,:,:] = NBR_composite[r1:r2,c1:c2]
    burnscar_composite = ncfile.createVariable('burnscarMask',np.float32,('latitude','longitude'), fill_value=-999) # note: unlimited dimension is leftmost
#     burnscar_composite.units = 'unitless' 
    burnscar_composite.long_name = 'burn_scar_mask'
    burnscar_composite[:] = burnscar_composite_with_timeDim
    

## Plot 2D Hists of NBR vs NDVI, BS RGB Channels

In [ ]:
# %matplotlib qt
fontsize=22
plt.rcParams['font.size'] = fontsize
plt.style.use('dark_background')
ylabels = ['NDVI', '2.25 micron', '0.86 micron', '0.67 microns']
fig_scat, ax_scat = plt.subplots(nrows=2, ncols=2, figsize=(24,13))
fig_scat.subplots_adjust(top=0.975, 
                    bottom=0.172,
                    left=0.196, 
                    right=0.844, 
                    hspace=0.207,
                    wspace=0.223)

r1,r2, c1,c2 = 378,563, 1527,1710
r1,r2, c1,c2 = 240,746, 1235,1836
r1,r2, c1,c2 = 424,440, 1586,1598
NBR_flat = NBR[r1:r2,c1:c2].flatten()
bins=100
cmap='jet'
vmin, vmax = 0,3
burn_scar_RGB_unscaled = burn_scar_RGB/2.5
ax_scat[0,0].hist2d(NBR_flat, NDVI[r1:r2,c1:c2].flatten(), bins=bins, cmap=cmap, vmin=vmin, vmax=vmax)
ax_scat[0,1].hist2d(NBR_flat, burn_scar_RGB_unscaled[r1:r2,c1:c2,0].flatten(), bins=bins, cmap=cmap, vmin=vmin, vmax=vmax)
ax_scat[1,0].hist2d(NBR_flat, burn_scar_RGB_unscaled[r1:r2,c1:c2,1].flatten(), bins=bins, cmap=cmap, vmin=vmin, vmax=vmax)
# im = ax_scat[1,1].hist2d(NBR_flat, burn_scar_RGB_unscaled[r1:r2,c1:c2,2].flatten(), bins=bins, cmap=cmap, vmin=vmin, vmax=vmax)

im = ax_scat[1,1].hist2d(burn_scar_RGB_unscaled[r1:r2,c1:c2,1].flatten(), burn_scar_RGB_unscaled[r1:r2,c1:c2,0].flatten(), bins=bins, cmap=cmap, vmin=vmin, vmax=vmax)
ax_scat[1,1].set_xlabel(ylabels[1], fontsize=fontsize)
ax_scat[1,1].set_ylabel(ylabels[2], fontsize=fontsize)

for i, a in enumerate(ax_scat.flat):
    if i!=3:
        a.set_xlabel("NBR", fontsize=fontsize)
        a.set_ylabel(ylabels[i], fontsize=fontsize)
    a.grid(linestyle='dashed', c='black', linewidth=2)

cb_ax = fig_scat.add_axes([0.15, 0.06, 0.7, 0.03]) #[left, bottom, width, height]
cbar = fig_scat.colorbar(im[3], cax=cb_ax, ticks = np.arange(vmin, vmax, 20),orientation="horizontal")
cbar.ax.tick_params(rotation=45)
# plt.tight_layout()    
plt.show()

## Plot Bs RGB Channel Images and 1D Hists with Interactive Threshold Sliders

In [ ]:
%matplotlib qt
from matplotlib.widgets import RangeSlider
plt.close()
burn_scar_RGB_unscaled = burn_scar_RGB/2.5
img = burn_scar_RGB_unscaled[r1:r2,c1:c2,:]
plt.style.use('dark_background')
fontsize=16
plt.rcParams['font.size'] = fontsize
fig, axs = plt.subplots(2, 4, figsize=(10, 5))
plt.subplots_adjust(bottom=0.25)


img_full = axs[0,0].imshow(2.5*img)
axs[1,0].axis('off')
for i, a in enumerate(axs.flat):
    a.set_xticks([])
    a.set_yticks([])
    if i==3:
        break

img_225 = img[:,:,0]
img_086 = img[:,:,1]
img_067 = img[:,:,2]

im_225 = axs[0,1].imshow(img[:,:,0])
axs[1,1].hist(img_225.flatten(), bins='auto')
axs[1,1].set_title('2.25 microns')
im_086 = axs[0,2].imshow(img[:,:,1])
axs[1,2].hist(img_086.flatten(), bins='auto')
axs[1,2].set_title('0.86 microns')
im_067 = axs[0,3].imshow(img[:,:,2])
axs[1,3].hist(img_067.flatten(), bins='auto')
axs[1,3].set_title('0.67 microns')

im_225.cmap.set_under('r')
im_225.cmap.set_over('r')
im_086.cmap.set_under('r')
im_086.cmap.set_over('r')
im_067.cmap.set_under('r')
im_067.cmap.set_over('r')

# Create the RangeSlider
slider_ax_225 = plt.axes([0.20, 0.03, 0.60, 0.03])#[left, bottom, width, height]
slider_225 = RangeSlider(slider_ax_225, "2.25 micron thresh", img_225.min(), img_225.max())
slider_ax_086 = plt.axes([0.20, 0.06, 0.60, 0.03])
slider_086 = RangeSlider(slider_ax_086, "0.86 micron thresh", img_086.min(), img_086.max())
slider_ax_067 = plt.axes([0.20, 0.09, 0.60, 0.03])
slider_067 = RangeSlider(slider_ax_067, "0.67 micron thresh", img_067.min(), img_067.max())

# Create the Vertical lines on the histogram
lower_limit_line_225 = axs[1,1].axvline(slider_225.val[0], color='r')
upper_limit_line_225 = axs[1,1].axvline(slider_225.val[1], color='r')
lower_limit_line_086 = axs[1,2].axvline(slider_086.val[0], color='r')
upper_limit_line_086 = axs[1,2].axvline(slider_086.val[1], color='r')
lower_limit_line_067 = axs[1,3].axvline(slider_067.val[0], color='r')
upper_limit_line_067 = axs[1,3].axvline(slider_067.val[1], color='r')




def update_225(val):
    # The val passed to a callback by the RangeSlider will
    # be a tuple of (min, max)

    # Update the image's colormap
    im_225.norm.vmin = val[0]
    im_225.norm.vmax = val[1]
    
    
    # Update the position of the vertical lines
    lower_limit_line_225.set_xdata([val[0], val[0]])
    upper_limit_line_225.set_xdata([val[1], val[1]])
    
    # Redraw the figure to ensure it updates
    fig.canvas.draw_idle()
    bs_RGB = np.copy(img)
    bs_RGB[bs_RGB<val[0] or bs_RGB>val[1]]=0
    
    return val

def update_086(val):    
    # Update the image's colormap
    im_086.norm.vmin = val[0]
    im_086.norm.vmax = val[1]

    # Update the position of the vertical lines
    lower_limit_line_086.set_xdata([val[0], val[0]])
    upper_limit_line_086.set_xdata([val[1], val[1]])
    
    # Redraw the figure to ensure it updates
    fig.canvas.draw_idle()
    bs_RGB = np.copy(img)
    bs_RGB[bs_RGB<val[0] or bs_RGB>val[1]]=0
    
    return val

def update_067(val):   
     # Update the image's colormap
    im_067.norm.vmin = val[0]
    im_067.norm.vmax = val[1]
    
    # Update the position of the vertical lines
    lower_limit_line_067.set_xdata([val[0], val[0]])
    upper_limit_line_067.set_xdata([val[1], val[1]])

    # Redraw the figure to ensure it updates
    fig.canvas.draw_idle()
    bs_RGB = np.copy(img)
    bs_RGB[bs_RGB<val[0] or bs_RGB>val[1]]=0
    
    return val

val_225 = slider_225.on_changed(update_225)
val_086 = slider_086.on_changed(update_086)
val_067 = slider_067.on_changed(update_067)
plt.show()

# NDSI and Snow RGB Composite Analysis for USNIC

## Get Ref and Geolocation for Snow Fall analysis

In [ ]:
# file_num = -1
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning) 
# test_ref_snow = "R:/VIIRS_data/test_case_snow/VJ102MOD.A2021077.1818.021.2021081151736.nc"
# # test_geo_snow = "R:/VIIRS_data/test_case_snow/"
# VJ102MOD.A2021077.1954.021.2021081151852.nc
# time_stamp_current = test_geo[-33:-21]
# lat, lon, SZA, VZA, SAA, VAA = get_VJ103_geo(test_geo_snow)

In [ ]:
# M_bands, M_band_solar_irradiances = get_VJ102_ref(test_ref_snow)

In [ ]:
import os
print(os.listdir("R:/VIIRS_data/test_case_snow"))

In [ ]:
# # %matplotlib qt
# %matplotlib inline
# plt.figure(figsize=(15,15))
# cosSZA = np.cos(np.deg2rad(SZA))
# print(M_bands[:5,:5,5])
# # M_bands[M_bands<0]==0
# M_bands_norm = np.copy(M_bands)
# print(np.nanmean(M_bands_norm))
# for i in range(16):
#     try:
#         M_bands_norm[:,:,i] /=  cosSZA
#     except:
#         M_bands_norm[:3216,:,i] /=  cosSZA[:3216,:]
# #     M_bands_norm[:,:,i] /=np.max(M_bands_norm[:,:,i])
    
# rgb= np.dstack((M_bands_norm[:,:,4], M_bands_norm[:,:,3], M_bands_norm[:,:,2]))
# rgb=np.flip(rgb, axis=0)
# rgb=2.5*np.flip(rgb, axis=1)
# plt.imshow(rgb)#, cmap='binary')
# # plt.imshow(np.flip(lat, axis=0), cmap='jet')
# # plt.colorbar()
# plt.show()

In [ ]:
def get_NDSI(R_M4, R_M10):
    #green and 1.6micron channels
    return (R_M4-R_M10)/(R_M4+R_M10)

def get_snowRGBcomposite(R_M4,R_M10,R_M11):
    #m4 could be m3 but using green over blue will reduce atmospheric scattering
    #the only down side is the slightly increased signal from vegetation...
    #green, 1.6 micron, 2.25 micron
    return np.dstack((R_M4,R_M10,R_M11))



## Snow RGB Composite

In [ ]:
# R_M11, R_M10, R_M4 = M_bands_norm[:,:,10], M_bands_norm[:,:,9], M_bands_norm[:,:,3]
# snow_RGB = 2.5*get_snowRGBcomposite(R_M11, R_M10, R_M4)
# snow_RGB_nan_idx = np.argwhere(np.isnan(snow_RGB)==True)
# print(len(snow_RGB_nan_idx[:,0]))
# # for i in snow_RGB_nan_idx[:,0]:
# #     print(i)
# #     for j in snow_RGB_nan_idx[:,1]:
# #         if j>0 and i>0 and j<len(snow_RGB[0]) and i<len(snow_RGB):
# #             pix_sum = np.nansum(snow_RGB[i+1,j+1,:],snow_RGB[i+1,j-1,:],\
# #                                 snow_RGB[i-1,j+1,:],snow_RGB[i-1,j-1,:])
# #             snow_RGB[i,j] = pix_sum/4

# snow_RGB=np.flip(snow_RGB, axis=0)
# snow_RGB=np.flip(snow_RGB, axis=1)
# plt.figure(figsize=(15,15))
# plt.imshow(snow_RGB)
# plt.show()


In [ ]:

# def interpolate_bowtie(arr):
#     '''
#     input:  3D numpy array to be modified
#     return: 3D numpy arr w/NaN values taking on the 
#             value of the average of the 4 pixels
#             touching the corner of the center pixel.
#             Where an average is not possible,
#             it will stay NaN.
#     '''
#     arr_nan_idx = np.argwhere(np.isnan(arr)==True)
    
#     # i+1, j+1
#     arr_nan_idx_upperright = np.copy(arr_nan_idx) 
#     arr_nan_idx_upperright[:][:2] += 1
    
#     # i-1, j-1
#     arr_nan_idx_lowerleft = np.copy(arr_nan_idx) 
#     arr_nan_idx_lowerleft[:][:2] -= 1
    
#     # i+1, j-1
#     arr_nan_idx_upperleft = arr_nan_idx_upperright[:][1] - 2
    
#     # i-1, j+1
#     arr_nan_idx_lowerright = arr_nan_idx_lowerleft[:][1] - 2
#     print(arr_nan_idx_lowerright)
#     arr[arr_nan_idx]= (arr[arr_nan_idx_lowerleft]+arr[arr_nan_idx_lowerleft]+\
#                        arr[arr_nan_idx_upperleft]+arr[arr_nan_idx_lowerright])/4
    
#     arr = arr[1:,1:,:]+arr[1:,1:,:]+arr[1:,1:,:]+arr[1:,1:,:]
    
#     return arr

# interpolate_bowtie(snow_RGB)
# plt.figure(figsize=(15,15))
# plt.imshow(snow_RGB)
# plt.show()

## NDSI

In [ ]:
# R_M10, R_M4 = M_bands_norm[:,:,9], M_bands_norm[:,:,3]
# NDSI = get_NDSI(R_M4, R_M10)
# NDSI =np.flip(NDSI, axis=0)
# NDSI =np.flip(NDSI, axis=1)
# plt.figure(figsize=(15,15))
# plt.imshow(NDSI, cmap='PiYG', vmin=-0.8, vmax=0.8)
# plt.colorbar()
# plt.show()
 

# Run all Composites on all VIIRS Files (then pick test cases)

In [ ]:
# file_num = 100
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning) 

# for geo_f, ref_f in zip(geo_filepaths, ref_filepaths) 
#     lat, lon, SZA, VZA, SAA, VAA = get_VJ103_geo(geo_filepaths[file_num])
#     #print time stamp to monitor progress
#     print(geo_filepaths[file_num][-33:-21])
#     M_bands, M_band_solar_irradiances = get_VJ102_ref(ref_filepaths[file_num])
    
#     #run composites and save to pdf